In [13]:
import torch
import torchvision
import torch.nn
import torch.optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score


# Check for available device (GPU, MPS, or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [14]:
from torch.utils.data import Subset
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data import random_split

data_root =  './train_data/'
image_size = (300,300)

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

train_dataset = ImageFolder(root=data_root, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size

train_dataset, test_dataset = random_split(train_dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)





# EDA

In [15]:
# Step 1: Conduct exploratory data analysis (EDA).
# By performing EDA, you can have a better understanding about the data, which helps you
# decide whether to perform pre-processing (such as image filtering or feature engineering) and
# which methods to perform in order to improve the data quality. Check they’re all the right
# size. 

class_names = train_dataset.dataset.classes
print(class_names)

class_counts = train_dataset.dataset.targets

for i in range(len(class_names)):
    print(f"Class {class_names[i]}: {class_counts.count(i)} images")	

['cherry', 'strawberry', 'tomato']
Class cherry: 1495 images
Class strawberry: 1495 images
Class tomato: 1495 images


# Baseline Model

In [16]:
# Step 3: Build a simple Baseline model.
# You should build a simple/standard neural network, i.e. multilayer perceptron (MLP) trained
# by back-propagation for this step, which helps you have a baseline intuition / idea of the task.
# It is probably easiest to just do this in PyTorch. Don’t use a CNN for this part! The point
# is to get a sense of how well a “vanilla” neural net can do, without convolutions. Because it
# is “dense”, this probably can’t be too large, as it has to be trained in a reasonable amount of
# time. Start small and see how you go.

class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
input_size = 300*300*3
hidden_size = 100
num_classes = 3

model = MLP(input_size, hidden_size, num_classes).to(device)

#Train the model
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum =0.9)

num_epochs = 10
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 300*300*3).to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}')

Epoch [1/10], Step [100/113], Loss: 1.019852876663208
Epoch [2/10], Step [100/113], Loss: 1.0805732011795044
Epoch [3/10], Step [100/113], Loss: 0.88969886302948
Epoch [4/10], Step [100/113], Loss: 0.7597952485084534
Epoch [5/10], Step [100/113], Loss: 0.7798830270767212
Epoch [6/10], Step [100/113], Loss: 0.7996910810470581
Epoch [7/10], Step [100/113], Loss: 0.5921481847763062
Epoch [8/10], Step [100/113], Loss: 0.5194516777992249
Epoch [9/10], Step [100/113], Loss: 0.7044929265975952
Epoch [10/10], Step [100/113], Loss: 0.6522991061210632


In [17]:
# Evaluate the Baseline model.

model.eval()  
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 300*300*3).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the test images: 47.38015607580825 %
